In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

dataset = pd.read_csv("./../../TushareStock/Stocks/t_000001", sep='\t',  names=["date", "open", "high","close","low","volume","price_change","p_change","ma5","ma10","ma20","v_ma5","v_ma10","v_ma20","turnover"], header=0)
print(dataset.head(5))

data = dataset[[ 'close', 'volume', 'open', 'high', 'low']].values.astype(float)
print(data[0:5, :])
data = np.flipud(data)

lookback_period = 30
data_matrix = np.empty([(data.shape[0] - lookback_period), data.shape[1], lookback_period])
# print(data_matrix)

scaler = MinMaxScaler(feature_range=(-1, 1))

for i in range(data_matrix.shape[0]): # for each example
    for j in range(data_matrix.shape[1]): # for each feature
        scaler.fit(data[i: i + lookback_period, j].reshape(lookback_period, 1))

data_matrix[i, j, :] = scaler.transform(data[i: i + lookback_period, j].reshape(1, -1))
data_matrix = np.swapaxes(data_matrix, 1, 2)
print(data_matrix[:5, :])

 # create y values: 1 if close at day 30 > close at day 29. Else 0.
def up_down(yest, tod):
    if tod >= yest:
        return 1
    else:
        return 0
perm = np.random.permutation(data_matrix.shape[0])
data_matrix = data_matrix[perm]
targets = np.empty([data_matrix.shape[0], 1])
for i in range(data_matrix.shape[0]):
    targets[i] = up_down(data_matrix[i][-2][0], data_matrix[i][-1][0])

         date   open   high  close    low      volume  price_change  p_change  \
0  2020-10-28  17.76  17.90  17.63  17.29  1205823.88         -0.13     -0.73   
1  2020-10-27  18.00  18.00  17.76  17.50  1034865.06          0.06      0.34   
2  2020-10-26  18.20  18.29  17.70  17.45  1175598.62         -0.43     -2.37   
3  2020-10-23  17.53  18.78  18.13  17.53  1698501.75          0.57      3.25   
4  2020-10-22  17.94  18.50  17.56  17.30  1890519.00         -0.35     -1.95   

      ma5    ma10    ma20       v_ma5      v_ma10      v_ma20  turnover  
0  17.756  17.537  16.488  1401061.66  1492172.26  1213566.33      0.62  
1  17.812  17.377  16.385  1408808.91  1437846.11  1196662.96      0.53  
2  17.768  17.207  16.290  1393850.29  1425241.56  1189727.79      0.61  
3  17.724  17.027  16.209  1561951.69  1466816.42  1199607.52      0.88  
4  17.518  16.732  16.081  1641374.19  1387008.84  1164093.21      0.97  
[[1.76300000e+01 1.20582388e+06 1.77600000e+01 1.79000000e+01
  1.729

In [32]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_matrix[:, :-1, :], targets, stratify=targets, test_size=0.2)
'''
layers: 1 LSTM (32 units)
1 Dense (1 unit)
lookback_period = 30
'''
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential()
model.add(LSTM(32, input_shape=(x_train.shape[1], x_train.shape[2]), stateful=False, return_sequences=True))
model.add(LSTM(8, input_shape=(x_train.shape[1], x_train.shape[2]), stateful=False))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss="binary_crossentropy", optimizer='adam', metrics = ['accuracy'])
EarlyStopping(monitor='val_acc', min_delta=0.001, patience=20)
model.fit(x_train, y_train, batch_size=20, validation_split = 0.20, epochs=100, shuffle=False)

 # baseline accuracy (= accuray if you always chose the most frequent y-value in testset)
baseline_acc= float(max(sum(y_test)/len(y_test), (1 - sum(y_test)/len(y_test))))
print("Baseline accuracy: " + str(baseline_acc))

# LSTM accuracy
loss_and_metrics = model.evaluate(x_test, y_test)
lstm_acc = float(loss_and_metrics[1])
print("LSTM accuracy: " + str(lstm_acc))
# LSTM premium
premium = lstm_acc - baseline_acc
print("LSTM premium: " + str( premium))
# a = pd.DataFrame.from_dict(baseline_acc, orient='index').rename(columns = {0: "baseline_acc"})
# b = pd.DataFrame.from_dict(lstm_acc, orient='index').rename(columns = {0: "lstm_acc"})
# c = pd.DataFrame.from_dict(premium, orient='index').rename(columns = {0: "premium"})
# result = pd.concat([a, b, c], axis=1)
# result

Train on 369 samples, validate on 93 samples
Epoch 1/100
369/369 [==============================] - 2s 5ms/step - loss: 0.6343 - acc: 0.9458 - val_loss: 0.5097 - val_acc: 1.0000
Epoch 2/100
369/369 [==============================] - 1s 1ms/step - loss: 0.2670 - acc: 1.0000 - val_loss: 0.0896 - val_acc: 1.0000
Epoch 3/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0574 - acc: 1.0000 - val_loss: 0.0342 - val_acc: 1.0000
Epoch 4/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0290 - acc: 1.0000 - val_loss: 0.0259 - val_acc: 1.0000
Epoch 5/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0246 - acc: 1.0000 - val_loss: 0.0233 - val_acc: 1.0000
Epoch 6/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0223 - acc: 1.0000 - val_loss: 0.0213 - val_acc: 1.0000
Epoch 7/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0204 - acc: 1.0000 - val_loss: 0.0195 - val_acc: 1.0000
Epoch 8/100
369/369 [===

Epoch 62/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 63/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 64/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000
Epoch 65/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000
Epoch 66/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 67/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 68/100
369/369 [==============================] - 1s 1ms/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 69/100
369/369 [==============================] - 1s 1ms